# Recipe Shopping List

This notebook generates a shopping list for recipe ingredient list specified in a spreadsheet saved as TSV (tab).

See the installation instructions for the [.NET Jupyter kernel](https://github.com/dotnet/interactive/blob/master/docs/NotebooksLocalExperience.md).

In [1]:
open System.IO
open System.Text.RegularExpressions

type RecipeIngredient =
    {
        Recipe : string
        Ingredient : string
        Quantity : float
        Unit : string
        Location : float
    } with 
    static member Create(recipe, ingredient, quantity, unit, location ) = { Recipe = recipe; Ingredient = ingredient; Quantity = quantity; Unit = unit; Location = location }

//matches a numeric decimal quantity followed by a non numeric unit identifier, e.g. "1.5oz"
let quantityRegex = new Regex( @"([0-9\.]+)\s*(\D+)")

//Map of recipe name to list of ingredients
let db = 
    Directory.GetFiles(".","*.tsv").[0] //will take the first file we find
    |> File.ReadAllLines
    |> Array.skip 1                     //skip header
    |> Array.map( fun row ->
        let s = row.Split('\t')
        
        //last col mixes quantity and unit
        let m = quantityRegex.Match(s.[2])
        
        //if we have quantity and unit (e.g. 1.5oz)
        if m.Groups.Count = 3 then
            { Recipe=s.[0]; Ingredient=s.[1]; Quantity=m.Groups.[1].Value|>float; Unit=m.Groups.[2].Value; Location = s.[3] |> float }
        //unit is identity (e.g. 1 carrot)
        else
            { Recipe=s.[0]; Ingredient=s.[1]; Quantity=s.[2]|>float; Unit=""; Location = s.[3] |> float}
    )
    |> Array.groupBy( fun ri -> ri.Recipe )
    |> Map.ofArray
    
//display the recipes we know about
let format ( s : string ) = s.Replace(";",";\n")
db |> Map.toArray |> Array.map fst |> sprintf "%A" |> format

The below script needs to be able to find the current output cell; this is an easy method to get it.

[|"baked ziti chickpeas";
 "cabbage cheddar pie";
 "celery curry";
 "chana masala";

  "cream of lentil soup";
 "cuban black beans";
 "edamame fried rice";

  "edamame noodles";
 "lentils with fruit and sweet potatoes";
 "okra curry";

  "penne all'arrabbiata";
 "portabello mushroom ragout";
 "potato chilli burro";

  "spinach enchiladas verde";
 "spinach lasagne";
 "spinach lentil soup";

  "sweet potato beetroot curry";
 "sweet potato black bean quessadillas";

  "sweet potato burritos";
 "szechuan tofu broccoli mushrooms";

  "two step southwestern stew";
 "veggie burros";
 "zucchini tostadas"|]

**Copy the list above below and remove the recipes you don't want**
(TODO checkbox GUI)

In [9]:
let recipes =
    [|"baked ziti chickpeas";
 //"cabbage cheddar pie";
 "celery curry";
 "chana masala";

"cream of lentil soup";
 "cuban black beans";
 //"edamame fried rice";

 "edamame noodles";
 "lentils with fruit and sweet potatoes";
 "okra curry";

  "penne all'arrabbiata";
//  "portabello mushroom ragout";
 //"potato chilli burro";

  "spinach enchiladas verde";
 //"spinach lasagne";
  "spinach lentil soup";

   "sweet potato beetroot curry";
 "sweet potato black bean quessadillas";

  //"sweet potato burritos";
 "szechuan tofu broccoli mushrooms";

//   "two step southwestern stew";
 "veggie burros";
 "zucchini tostadas"|]
//-----------------------------
// given selected recipes, collect all ingredients as groups, and sum their quantities
let ingredients =
    recipes
    
    //index by ingredient
    |> Array.collect( fun r ->
        db.[r] |> Array.map( fun ri -> ri.Ingredient,ri )
    )
    |> Array.sortBy( fun (ingredient,ri) -> ri.Location )
    |> Array.groupBy fst

    //aggregate quantities for each ingredient as much as possilbe
    |> Array.map( fun (i,tuples) -> 
    
        //sum quantities of the same unit
        let unitQuantities = 
            tuples
            |> Array.map snd
            |> Array.groupBy( fun ri2 -> ri2.Unit ) 
            |> Array.map( fun (u,ris2) ->
                (ris2 |> Array.sumBy( fun ri2 -> ri2.Quantity ) |> string) + " " + u
            )
        
        //map to string
        i + ":" + (unitQuantities |> String.concat "; ")
    )
    
printfn "%A" (ingredients |> String.concat "\n")
printfn "-----------"
printfn "%A" (recipes |> String.concat "\n")

"ginger:6 in; 4 tb
sweet potato:5 ; 1 large
lemon:2 
onion:7 
banana:3 
green bell pepper:2 
garlic:25 clove
green onion:11 
green onions:7 
lime:1 
spinach:38 oz
tomatoes:3 can
pear:2 
plantain:1 
okra:6 c
cherry tomatoes:8 oz
carrot:6 
cremini mushrooms:8 oz
zucchini:7 
cauliflower:1 
celery:1 
broccoli:1 c; 1 
cilantro:0.5 c
chillis:2 
tomatoes green chillies:3 can
diced tomatoes:1 can
kidney beans:1 can
tortillas:23 
salsa:2 c
penne pasta:24 oz
green chillis:4 oz
crushed tomatoes:1 can
beetroot:1 can
green salsa:7 oz
chili paste with garlic:1 tb
soy sauce:0.5 c
cooking sherry:0.25 c
salsa verde:1 
stock cubes:3 
black beans:5 can
lentils:2.5 c; 16 oz
kalamata olives:0.25 c
chipotle adobo sauce:1 t
chickpeas:3 can
cream of mushroom soup:12 oz
rice:4 c
rice vinegar:2 tb
tomato puree:1 c
udon noodles:9 oz
tomatoes basil garlic oregano:1 can
pineapple:1 can
tomato paste:1 tb
vegetable oil:20 tb
oregano:1 t
garam masala:1 t
cumin:3.5 t; 1 tb
garlic salt:0.5 t
breadcrumbs:2 tb
corn starc